In [1]:
import pandas as pd 
import numpy as np
import nlp.wordbags as wb 
from nlp.word2vec import Word2WordPrediction
from nlp.w2w_readers import WordEmbeddings
import pickle


## Esempio minimo di Word2Vec sia come predittore che come embedding 

In [2]:
corpus = [
    ['montagna', 'neve', 'ghiaccio', 'vetta', 'bosco', 'freddo'],
    ['spiaggia', 'sabbia', 'mare', 'onda', 'sole', 'caldo'],
    ['deserto', 'duna', 'cactus', 'sabbia', 'sole', 'arido'],
    ['foresta', 'tropicale', 'liana', 'sole', 'umido', 'caldo'],
    ['lago', 'abete', 'vento', 'neve', 'bosco', 'freddo'],
    ['ghiaccio', 'artico', 'tormenta', 'neve', 'iceberg', 'gelido'],
    ['duna', 'oasi', 'deserto', 'cactus', 'arido']
]

In [3]:
bow = wb.Bow(corpus=corpus)
data_loader, inputs, outputs = bow.one_hot_skip_gram_dataloader(window=4, batch=2)

In [4]:
for doc in corpus:
    print(" | ".join(doc))

montagna | neve | ghiaccio | vetta | bosco | freddo
spiaggia | sabbia | mare | onda | sole | caldo
deserto | duna | cactus | sabbia | sole | arido
foresta | tropicale | liana | sole | umido | caldo
lago | abete | vento | neve | bosco | freddo
ghiaccio | artico | tormenta | neve | iceberg | gelido
duna | oasi | deserto | cactus | arido


## Predittore semplice

### Initialize and training the model

In [18]:
w2w = Word2WordPrediction(input_dim=bow.size, hidden_dim=4)
epochs = 50
learning_rate = 0.001
history = w2w.train(data_loader=data_loader, epochs=epochs, 
                    learning_rate=learning_rate)
print(history[0], history[-1])

100%|██████████| 50/50 [00:00<00:00, 309.33it/s]

15.233291580563499 15.119768324352446


In [19]:
model = WordEmbeddings(words=bow, model=w2w)

### Predizione

In [20]:
search = 'deserto'
y_pred = model.predict(search, topk=10)
y_pred

cactus      0.152648
duna        0.119723
sabbia      0.089247
oasi        0.084072
sole        0.069445
mare        0.054596
deserto     0.047798
arido       0.038609
ghiaccio    0.031109
neve        0.026679
dtype: float32

### Pseudo generazione

In [21]:
start = "iceberg"
testo = [start]
for i in range(6):
    p = model.predict(start)
    next_word = np.random.choice(p.index.values, p=p.values / p.values.sum())
    testo.append(next_word)
    start = next_word

In [22]:
testo

['iceberg', 'neve', 'freddo', 'duna', 'cactus', 'sole', 'tropicale']

### Embeddings

In [13]:
model[search]
model['montagna']

array([ 0.82403773,  0.81757665,  1.0569373 , -0.75357807], dtype=float32)

In [16]:
search = "deserto"
y_sim = model.most_similar(search, topk=10)
y_sim

deserto    1.000000
arido      0.995039
sabbia     0.751156
cactus     0.661013
sole       0.529758
mare       0.418151
oasi       0.366112
caldo      0.328773
onda       0.302740
duna       0.267332
Name: deserto, dtype: float32

## Compositionality

In [ ]:
answer, a_vector = model.analogy('deserto', 'duna', 'mare')
print(answer)

In [23]:
model.common_meanings(words=['bosco', 'foresta', 'montagna', 'freddo'])

freddo      0.999315
ghiaccio    0.955680
bosco       0.863940
lago        0.837324
montagna    0.811669
vetta       0.763923
abete       0.673722
vento       0.624856
neve        0.621306
tormenta    0.339243
dtype: float32

In [24]:
model.spot_odd_one(words=['montagna', 'vetta', 'ghiaccio', 'sabbia'])

sabbia      0.063810
ghiaccio    0.695018
vetta       0.950431
montagna    0.956933
dtype: float32

In [ ]:
model.search(positive=['deserto', 'mare'], negative=['duna'])

In [ ]:
model.search(positive=['mare'], negative=['caldo'])